In [52]:
import pandas as pd
import socket
import numpy as np
import math
import gc
from collections import defaultdict

In [61]:
jdata_action_train = pd.read_hdf('./data/action_train_with_neg_goodsex.h5', key='df', mode='r')
# jdata_action_test = pd.read_hdf('./data/action_test_with_neg_goodsex.h5', key='df', mode='r')
jdata_comment = pd.read_csv('./raw/jdata_comment.csv',sep=',')
jdata_product = pd.read_csv('./raw/jdata_product.csv',sep=',')
jdata_shop = pd.read_csv('./raw/jdata_shop.csv',sep=',')
jdata_user = pd.read_csv('./raw/jdata_user.csv',sep=',')

In [54]:
# jdata_action_train = jdata_action_train[:-5000000]
jdata_action_train = jdata_action_train.merge(jdata_product[['sku_id','brand','shop_id','cate']], on='sku_id', how='left')
jdata_action_train = jdata_action_train.merge(jdata_user[['user_id','age','city']], on='user_id', how='left')
jdata_action_train= jdata_action_train.merge(jdata_shop[['shop_id','fans_num','shop_score']], on='shop_id', how='left')
# jdata_action_train= jdata_action_train.merge(jdata_comment[['sku_id','comments']], on='sku_id', how='left')
jdata_action_train =jdata_action_train.fillna(0) 

In [4]:
# jdata_action_test = jdata_action_test[:-1000000]
jdata_action_test = jdata_action_test.merge(jdata_product[['sku_id','brand','shop_id','cate']], on='sku_id', how='left')
jdata_action_test = jdata_action_test.merge(jdata_user[['user_id','age','city']], on='user_id', how='left')
jdata_action_test= jdata_action_test.merge(jdata_shop[['shop_id','fans_num','shop_score']], on='shop_id', how='left')
# jdata_action_train= jdata_action_train.merge(jdata_comment[['sku_id','comments','good_comments','bad_comments']], on='sku_id', how='left')
jdata_action_test = jdata_action_test.fillna(0) 

In [58]:
import lightgbm as lgb
jdata_action_train = jdata_action_train.sample(frac=1).reset_index(drop=True)
label = jdata_action_train['label']
jdata_action_train = jdata_action_train[['label','age','city','brand','shop_id','cate','fans_num','shop_score','user_id','sku_id']]
param = {'num_leaves': 100, 
         'objective': 'binary',
         'metric': ['auc', 'binary_logloss']
        }
train_data = lgb.Dataset(jdata_action_train, label=label, feature_name=['label','age','city','brand','shop_id','cate','fans_num','shop_score','user_id','sku_id'], categorical_feature=['age','city','brand','shop_id','cate','user_id','sku_id'])

In [59]:
num_round = 1
lgb.cv(param, train_data, num_round, nfold=5)

/opt/python3/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


{'binary_logloss-mean': [0.5785541791023128],
 'binary_logloss-stdv': [2.3061790360440876e-08],
 'auc-mean': [1.0],
 'auc-stdv': [0.0]}

In [57]:
print('Optimized memory....')
cols_int8 = ['label','age']
cols_int16 = ['city','cate']
cols_int32 = ['shop_id','brand','user_id','sku_id','fans_num']
cols_int64 = []
for col in cols_int8:
    jdata_action_train[[col]] = jdata_action_train[[col]].astype('int8', errors='ignore')
for col in cols_int16:
    jdata_action_train[[col]] = jdata_action_train[[col]].astype('int16', errors='ignore')
for col in cols_int32:
    jdata_action_train[[col]] = jdata_action_train[[col]].astype('int32', errors='ignore')
for col in cols_int64:
    jdata_action_train[[col]] = jdata_action_train[[col]].astype('int64', errors='ignore')

### 转换格式，拿到cate_memo
valid_cols = ['label','age','city','brand','shop_id','cate','fans_num','shop_score','user_id','sku_id']
jdata_action_train = jdata_action_train[valid_cols]

# df2libffm(jdata_action_train, y='label', save_file='./data/train_ffm_model.txt', non_categorical_cols = ['fans_num','shop_score'])

Optimized memory....


In [1]:
print('Optimized memory....')
cols_int8 = ['label','age']
cols_int16 = ['city','cate']
cols_int32 = ['shop_id','brand','user_id','sku_id']
cols_int64 = []
for col in cols_int8:
    jdata_action_test[[col]] = jdata_action_test[[col]].astype('int8', errors='ignore')
for col in cols_int16:
    jdata_action_test[[col]] = jdata_action_test[[col]].astype('int16', errors='ignore')
for col in cols_int32:
    jdata_action_test[[col]] = jdata_action_test[[col]].astype('int32', errors='ignore')
for col in cols_int64:
    jdata_action_test[[col]] = jdata_action_test[[col]].astype('int64', errors='ignore')

### 转换格式，拿到cate_memo
valid_cols = ['label','age','city','brand','shop_id','cate','fans_num','shop_score','user_id','sku_id']
jdata_action_test = jdata_action_test[valid_cols]

df2libffm(jdata_action_test, y='label', save_file='./data/test_ffm_model.txt', non_categorical_cols = ['fans_num','shop_score'])

In [5]:
import datetime
def df2libffm(df, save_file=None, y=None, non_categorical_cols = []) :
    """
    df: data source, should be pandas dataframe
    save_file: save file name, if None, will skip saving action
    y: label columns name in the pandas dataframe
    non_categorical_cols: columns in this list will not be one hot encoder. 
       example: 
           price : 0.99$
           click rate: 0.03
    """
    assert y is not None

    row_cnt = df.shape[0]

    out = pd.DataFrame({
        y: df[y]
    })

    print("out.shape:", out.shape)
    df = df.drop(columns=[y], axis=1)

    feature_base = 0
    

    for idx, col in enumerate(df.columns.tolist()) :
        dt = datetime.datetime.now()
        print( str(dt), idx, col)

        cur_field_id = idx
        field_series = pd.Series([cur_field_id] * row_cnt).astype(str)

        if col in non_categorical_cols :
            # if not categorical feature, do not consider how many different values
            feature_series = pd.Series([feature_base] * row_cnt).astype(str)
            feature_base += 1
            value_series = df[col].astype(str)

            new_col = field_series + ":" + feature_series + ":" + value_series
            out[str(cur_field_id)] = new_col.values
        else :
            # if is categorical feature
            ### 使用s.astype(cate_type)进行转换
            
            ## 为每一个category特征编号
            feature_series = df[col].astype('category').values.codes  
            ## 编号加上base（前一列的最大编号）
            feature_series = feature_series + feature_base
            feature_series = pd.Series(feature_series).astype(str)
            ## 更新base
            feature_base += feature_series.unique().shape[0]
            print("next feature base:", feature_base)
            
            new_col = field_series + ":" + feature_series + ":1" 
            out[str(cur_field_id)] = new_col.values
            

    if save_file:
        file_name = save_file
        if not file_name.endswith(".txt"):
            file_name += ".txt"
        print("save file name:", file_name)
        out.to_csv(file_name, sep=" ", header=False, index=False)
    else:
        return out